In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

/opt/miniconda3/envs/rag/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [21]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name='rag-index', dimension=3072, metric='cosine', spec=ServerlessSpec(cloud="aws", region='us-east-1')
)


In [22]:
import json
data = json.load(open('reviews.json'))
data['reviews']


[{'professor': 'Dr. Emily Johnson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Johnson is an excellent professor. Her lectures are engaging and she explains complex topics clearly.'},
 {'professor': 'Prof. Michael Lee',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Lee is very knowledgeable. His classes are challenging but rewarding if you put in the effort.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Dr. Thompson is passionate about biology, but sometimes her lectures can be a bit too fast-paced.'},
 {'professor': 'Prof. James Anderson',
  'subject': 'Physics',
  'stars': 2,
  'review': 'Prof. Anderson knows his stuff, but his teaching style is not very engaging. The material is hard to follow.'},
 {'professor': 'Dr. Laura Martinez',
  'subject': 'Chemistry',
  'stars': 4,
  'review': 'Dr. Martinez is very approachable and willing to help outside of class. Her exams are fair.'},
 {'professor': 'Prof. Da

In [30]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    combined_text = f"{review['review']} {review['subject']}"
    response_data = client.embeddings.create(
        input=combined_text,
        model='text-embedding-3-small'
    )
    embedding = response_data.data[0].embedding
    processed_data.append({
        'values': embedding,
        'id': review["professor"],
        'metadata': {
            'review': review["review"],
            'subject': review["subject"],
            'stars': review["stars"]
        }
    })

In [28]:
processed_data[1]

{'values': [-0.0095464885,
  0.01566211,
  -0.019641407,
  0.016784474,
  0.023926804,
  0.021898894,
  0.019488357,
  0.0065620155,
  -0.012843439,
  -0.002284589,
  0.007856563,
  0.019768948,
  -0.016593162,
  0.011070611,
  0.014271906,
  -0.003182163,
  0.02657967,
  0.01336636,
  -0.08458559,
  -0.018340483,
  -0.03359446,
  -0.048873942,
  0.012212109,
  0.04530278,
  -0.043797787,
  -0.0063388175,
  0.014144364,
  -0.017855825,
  0.0022877776,
  0.03435971,
  0.044256937,
  0.03142625,
  0.016912017,
  -0.0023977822,
  -0.0010314927,
  -0.017587988,
  0.0057776347,
  0.012059059,
  -0.04208873,
  0.029207027,
  -0.03163032,
  -0.02504917,
  -0.025470058,
  -0.015534568,
  0.053210355,
  -0.023340113,
  0.0015105709,
  -0.031400744,
  0.018774124,
  0.011236415,
  -0.03660444,
  0.012365159,
  0.064127915,
  -0.004569178,
  -0.012192978,
  0.01732015,
  0.007244363,
  -0.016159521,
  0.027242886,
  0.0012443278,
  0.048006658,
  0.015547321,
  -0.04466507,
  -0.021082627,
  0.03

In [33]:
movies = json.load(open('app/_data/embeddings.json'))

In [37]:
index = pc.Index('movies')
index.upsert(
    vectors=movies,
    namespace='ns1'
)

KeyboardInterrupt: 

In [ ]:
index.describe_index_stats()

MaxRetryError: HTTPSConnectionPool(host='movies-cv6nhkq.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /describe_index_stats (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1124c7ce0>: Failed to resolve 'movies-cv6nhkq.svc.aped-4627-b74a.pinecone.io' ([Errno 8] nodename nor servname provided, or not known)"))